## Import Requisite Lbraries

In [ ]:
import pandas as pd
import os  # import operating system for dir

## Ensure Directory

In [ ]:
from eda_toolkit import ensure_directory

base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to parent directory,
# then into the 'data' folder
data_path = os.path.join(os.pardir, "data")
data_output = os.path.join(os.pardir, "data_output")

# create image paths
image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure'data' directory exists
ensure_directory(data_path)
ensure_directory(data_output)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

## UCI ML Repository

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# Combine X and y into entire df
df = X.join(y, how="inner")

## Add Ids

In [ ]:
from eda_toolkit import add_ids

# Add a column of unique IDs with 9 digits and call it "census_id"
df = add_ids(
    df=df,
    id_colname="census_id",
    num_digits=9,
    seed=111,
    set_as_index=True,
)

df.head()

In [ ]:
if df.index.is_unique:
    print("The index is unique.")
else:
    print("The index is not unique.")

## Save Out the DataFrame

In [ ]:
df.to_csv(os.path.join(data_path, "adult_income.csv"))
df.to_parquet(os.path.join(data_path, "adult_income.parquet"))

## Binning Numerical Columns

In [ ]:
bin_ages = [
    0,
    18,
    30,
    40,
    50,
    60,
    70,
    80,
    90,
    100,
    float("inf"),
]

In [ ]:
label_ages = [
    "< 18",
    "18-29",
    "30-39",
    "40-49",
    "50-59",
    "60-69",
    "70-79",
    "80-89",
    "90-99",
    "100 +",
]

In [ ]:
df["age_group"] = pd.cut(
    df["age"],
    bins=bin_ages,
    labels=label_ages,
    right=False,
)

## Consolidate Income (Outcome) Column

In [ ]:
def outcome_merge(val):
    if val == "<=50K" or val == "<=50K.":
        return "<=50K"
    else:
        return ">50K"

In [ ]:
df["income"] = df["income"].apply(outcome_merge)

In [ ]:
df["income"].value_counts()

## Generate Table 1

In [ ]:
from eda_toolkit import generate_table1

table1 = generate_table1(
    df=df,
    value_counts=True,
    max_categories=3,
    export_markdown=True,
    markdown_path=os.path.join(data_output, "table1_summary.md"),
)

table1 = table1.drop(columns=["Type", "Mode"])
table1.to_clipboard()

In [ ]:
table1

In [ ]:
print(table1)

In [ ]:
from eda_toolkit import generate_table1

# Get DataFrame and Markdown string
df1 = generate_table1(
    df,
    value_counts=True,
    decimal_places=0,
)

In [ ]:
# Just get markdown string (no tuple)
md_only = generate_table1(
    df,
    export_markdown=True,
    return_markdown_only=True,
    combine=False,
    include_types="continuous",
    markdown_path=os.path.join(data_output, "table1_summary.md"),
)

In [ ]:
table1 = generate_table1(
    df,
    export_markdown=True,
    combine=False,
    include_types="categorical",
    markdown_path=os.path.join(data_output, "table1_summary.md"),
)

In [ ]:
# Save Markdown to file
generate_table1(
    df,
    value_counts=True,
    export_markdown=True,
    markdown_path=os.path.join(data_output, "custom_prefix_table1_summary.md"),
    combine=False,
)

In [ ]:
# Get DataFrame and Markdown string
df2 = generate_table1(
    df,
    value_counts=True,
    include_types="categorical",
    # decimal_places=0,
    combine=True,
)

In [ ]:
df2

In [ ]:
print(df2)

In [ ]:
df3, df4 = generate_table1(
    df,
    value_counts=True,
    include_types="both",
    max_categories=3,
    export_markdown=True,
    combine=False,
    markdown_path=os.path.join(data_output, "both_table1_summary.md"),
)

df3 = df3.drop(columns=["Type", "Mode"])

In [ ]:
df3

In [ ]:
print(df3)

In [ ]:
df3.to_clipboard()

In [ ]:
df4

In [ ]:
print(df4)

In [ ]:
df4

### Table 1 with P-Values

In [ ]:
# 1) Equal-width bins

df["fnlwgt_bin"] = pd.cut(
    df["fnlwgt"], bins=5, labels=[f"Bin {i}" for i in range(1, 6)], include_lowest=True
)

# 2) Equal-frequency (quantile) bins
df["fnlwgt_bin_quantile"] = pd.qcut(
    df["fnlwgt"],
    q=5,
    labels=[f"Q{i}" for i in range(1, 6)],
    duplicates="drop",  # in case of ties
)

In [ ]:
df_table1_cat = df[["fnlwgt_bin", "age_group", "income"]]

In [ ]:
p_value_table_1_cat = generate_table1(
    df_table1_cat,
    value_counts=True,
    include_types="categorical",
    export_markdown=True,
    groupby_col="income",
    drop_columns=["Missing (n)", "Missing (%)", "income", "Type", "Mode"],
    drop_variables="income",
    markdown_path=os.path.join(data_output, "p_val_table1_summary.md"),
)

In [ ]:
p_value_table_1_cat

In [ ]:
print(p_value_table_1_cat)

## Trailing Period Removal

In [ ]:
from eda_toolkit import strip_trailing_period

# Create a sample dataframe with trailing periods in some values
data = {
    "values": [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
}
df_trail = pd.DataFrame(data)

# Remove trailing periods from the 'values' column
df_trail = strip_trailing_period(df=df_trail, column_name="values")
df_trail

## Standardized Dates

In [ ]:
from eda_toolkit import parse_date_with_rule

# Sample date strings
date_strings = ["15/04/2021", "04/15/2021", "01/12/2020", "12/01/2020"]

# Standardize the date strings
standardized_dates = [parse_date_with_rule(date) for date in date_strings]

print(standardized_dates)

In [ ]:
# Creating the DataFrame
data = {
    "date_column": [
        "31/12/2021",
        "01/01/2022",
        "12/31/2021",
        "13/02/2022",
        "07/04/2022",
    ],
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "amount": [100.0, 150.5, 200.75, 250.25, 300.0],
}

df_fake = pd.DataFrame(data)

# Apply the function to the DataFrame column
df_fake["standardized_date"] = df_fake["date_column"].apply(parse_date_with_rule)

print(df_fake)

## DataFrame Analysis

In [ ]:
from eda_toolkit import dataframe_profiler

dataframe_profiler(df=df, background_color="brown")

## Generating Summary Tables for Variable Combinations

In [ ]:
from eda_toolkit import summarize_all_combinations

# Define unique variables for the analysis
unique_vars = [
    "age_group",
    "workclass",
    "education",
    "occupation",
    "race",
    "sex",
    "income",
]

# Generate summary tables for all combinations of the specified variables
summary_tables, all_combinations = summarize_all_combinations(
    df=df,
    # data_path=data_output,
    variables=unique_vars,
    # data_name="census_summary_tables.xlsx",
)

print(all_combinations)

## Saving DataFrames to Excel with Customized Formatting

In [ ]:
from eda_toolkit import save_dataframes_to_excel

# Example usage
file_name = "df_census.xlsx"  # Name of the output Excel file
file_path = os.path.join(data_path, file_name)

# filter DataFrame to Ages 18-40
filtered_df = df[(df["age"] > 18) & (df["age"] < 40)]

df_dict = {
    "original_df": df,
    "ages_18_to_40": filtered_df,
}

save_dataframes_to_excel(
    file_path=file_path,
    df_dict=df_dict,
    decimal_places=0,
)

## Creating Contingency Tables

In [ ]:
from eda_toolkit import contingency_table

# Example usage
contingency_table(
    df=df,
    cols=[
        "age_group",
        "workclass",
        "race",
        "sex",
    ],
    sort_by=1,
)

## Highlighting Specific Columns in a DataFrame

In [ ]:
from eda_toolkit import highlight_columns

# Applying the highlight function
highlighted_df = highlight_columns(
    df=df.head(),
    columns=["age", "education"],
    color="brown",
)

highlighted_df